# R-NET TBI Inference
## Intro
Given a list of nii.gz files inside a folder, makes a brain mask and a ROI mask prediction with a given loaded model

## Libraries

In [3]:
# utils
from utils.utils import save_excel_table
from utils.nifti import estimate_volume

# visualization
from utils.vedo import plot_slicer_cloud, plot_volume_cloud

# neural imaging
import nibabel as nib

# tensorflow
import tensorflow as tf
from evaluation.metrics import *

# other
import os
import importlib
import numpy as np

# make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


## Configuration
Here it's possible to change some paramaters to make predictions


Paths:
- dataset_folder (string): path to the folder that contains the files on which we want to make predictions
- save_folder (string): path to the folder in which we want to save the predictions

In [4]:
dataset_folder = '../data_fix/Caen_glunomab'
dataset_folder = r'\\calypso.marionegri.it\\ImagingUserData\\DannoCerebraleAcuto_MRI\\'
save_folder = dataset_folder

Save results:
- save (boolean): if true, saves the predictions in save_folder

In [5]:
save = True

3D preview:
- show_3d_preview (boolean): shows the predicted mask in an external window

In [6]:
show_3d_preview = False

Morphological smoothing:
- remove_small_objects (boolean): if true, removes small unconnected regions based on object_min_area
- object_min_area (int): the smallest allowable contiguos region size, in voxels
- fill_small_holes (boolean): if true, fills small holes
- holes_max_area (int): the maximum area, in voxels, of a contiguous hole that will be filled

In [5]:
remove_small_objects = True
object_min_area = 30000
fill_small_holes = True
holes_max_area = 20000

Prediction parameters:
- patch_size (tuple): size of the sliding window used to extract patches from the image
- patch_resolution (tuple): desired target resolution for all patch (should be equal to the training resolution of the model)
- stride (int): translation offset of the sliding window (less is better but requires more computational time)

Suggested stride values: 6,8,12,16

In [6]:
patch_size = (80,80,80) #voxels
patch_resolution = (0.1,0.1,0.1) #mm
stride = 20
threshold = 0.55

Input and output filenames:

In [7]:
# Input Image
input_postfix = '_anat_orig.nii.gz'

# Output Masks
brain_prediction_postfix = '_brain_mask_r3dnet.nii.gz'
roi_prediction_postfix = f'_regions_r3dnet.nii.gz'

# Excel Table Name
excel_name = f'predicted_r3dnet_volumes_dti.xlsx'

To make a prediction, the program expects a standard folder structure similar to the following:
```python
target_folder/
  FLASH/
    ID_MICE_A/
      Anat/
        ID_MICE_A_N4.nii.gz
        ...
    ID_MICE_B/
      Anat/
        ID_MICE_B_N4.nii.gz
        ...
  DTI/
    ID_MICE_F/
      dti/
        ID_MICE_F_N4.nii.gz
        ...
  Other.../
```
In this structure, there is a folder for each group of mice, such as `FLASH`, `RARE`, or `Group_1`. 

Inside each group folder, it is **strictly required** that each mouse file name starts with the same folder name as that of the mouse. 

For example, the `ID_MICE_A` folder should contain only files starting with `ID_MICE_A`. The subfolder (e.g., `Anat`) and the postfix file name extension (e.g., `N4`) can be modified later if needed.

You can specify the modalities directories in the following variable:

```python
modalities_directories = {
    'FLASH': 'Anat',
    'RARE': 'Anat',
    'DTI': 'dti',
    # Add more modalities as needed
}
```

In [8]:
# Folder structure for structured mode
modalities_directories = {
    'Biomech_dec24': 'Anat',
}

# Output labels ( Network -> Output mask labels)
labels_mapping = {
     0: 0,
     1: 1,
     2: 3,
     3: 21
 }

postprocessed_mapping = {
    0: 0,
    1: 1,
    2: 3,
    3: 13,
    4: 21
}

name_mapping = {
    0: {'name': 'Background', 'value': 0},
    1: {'name': 'Lesion', 'value': 1},
    2: {'name': 'Contra Ventricle', 'value': 3},
    3: {'name': 'Ipsi Ventricle', 'value': 13},
    4: {'name': 'Third Ventricle', 'value': 21}
}

num_classes = len(postprocessed_mapping)

From here the code should remain unchanged

In [9]:
num_classes = 4
input_channels = 1

# MODEL_NAME = 'mice_roi_unet_ep500_29-05-2024_00-19' #normal dice no smooth
# MODEL_NAME = 'mice_roi_unet_ep500_30-05-2024_11-29' # Long rat
# MODEL_NAME = 'mice_roi_unet_ep500_01-06-2024_20-06' # Inception v1
# MODEL_NAME = 'mice_roi_unet_ep500_02-06-2024_01-06' # Inception v2
# MODEL_NAME = 'mice_roi_unet_ep500_03-06-2024_03-39' # Super zoom
# MODEL_NAME = 'mice_roi_unet_ep500_03-06-2024_22-49' # Focal dice
# MODEL_NAME = 'mice_roi_unet_ep500_04-06-2024_08-00' # Tversky ce
# MODEL_NAME = 'mice_roi_unet_ep500_05-06-2024_10-10' # Corrected Loss riuscito
# MODEL_NAME = 'mice_roi_unet_ep500_05-06-2024_20-29' # Corrected Loss riuscito con ventricolo focal 2
# MODEL_NAME = 'mice_roi_unet_ep500_06-06-2024_12-29' # Corrected Loss non riuscito con ventricolo focal 3
# MODEL_NAME = 'mice_roi_unet_ep500_07-06-2024_17-27' # Changed random scalen changed a bit the loss 0.7-0.3

# MODEL_NAME = 'mice_roi_unet_ep500_17-06-2024_20-49'
# MODEL_NAME = 'multi_task_k_fold_ep500_23-06-2024_00-58'

MODEL_NAME = 'rats_rnet_pretrained'
MODEL_NAME = 'rnet_mice.h5'
MODEL_NAME = 'mice_roi_unet_ep500_07-11-2024_09-36'
full_name = 'model_fold_0.h5'
full_name = 'save_' + MODEL_NAME + '.h5'
model_path = '../results/' + MODEL_NAME + '/' + full_name

model_path = '../models/' + MODEL_NAME 
model_path = '../results/' + MODEL_NAME + '/' + full_name

In [10]:
# process_id = [ 'TBI_MC_fp_22_125', 'TBI_MC_fp_22_131','TBI_MC_fp_22_141','TBI_MC_fp_22_158',
#     'TBI_MC_fp_22_159', 'TBI_MC_fp_22_164', 'TBI_MC_fp_22_168', 'TBI_MC_fp_22_171', 'TBI_MC_fp_22_175', 'TBI_MC_fp_22_182', 
#     'TBI_MC_fp_22_183', 'TBI_MC_fp_22_185', 'TBI_MC_fp_22_188', 'TBI_MC_fp_22_194', 
#     'TBI_MC_fp_22_160', 'TBI_MC_fp_22_162', 'TBI_MC_fp_22_166', 'TBI_MC_fp_22_172', 'TBI_MC_fp_22_176', 
#     'TBI_MC_fp_22_178', 'TBI_MC_fp_22_190', 'TBI_MC_fp_22_192', 'TBI_MC_fp_22_193', 'TBI_MC_fp_22_163', 
#     'TBI_MC_fp_22_167', 'TBI_MC_fp_22_169', 'TBI_MC_fp_22_170', 'TBI_MC_fp_22_174', 'TBI_MC_fp_22_179', 
#     'TBI_MC_fp_22_181', 'TBI_MC_fp_22_186', 'TBI_MC_fp_22_189', 'TBI_MC_fp_22_195'
# ]

process_id =None

## Load the model
Load a previously trained model to start making predictions

In [11]:
import models.networks
from evaluation.metrics import *
from evaluation.losses import *

model = tf.keras.models.load_model(model_path,
                                   custom_objects={ "loss": diceCELoss(),
                                                    "precision": precision_coefficient(),
                                                    "sensitivity": sensitivity_coefficient(),
                                                    "specificity": specificity_coefficient(),
                                                    "K": tf.keras.backend,
                                                    "training": False,
                                                  }, compile=False)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 None, 1)]                                                        
                                                                                                  
 conv3d_10 (Conv3D)             (None, None, None,   448         ['input_1[0][0]']                
                                None, 16)                                                         
                                                                                                  
 batch_normalization_6 (BatchNo  (None, None, None,   64         ['conv3d_10[0][0]']              
 rmalization)                   None, 16)                                                     

## Preprocessing configuration
The standard pipeline order is:
1. (opt.) Cut and pad the image to a default matrix dimension
2. (opt.) Correct x10 intensity values
3. Apply N4 Bias Field correction
4. Copy the orientation from the ref. image
5. Resample to a target resolution
6. Normalize the intensity values with z-score (default)

In [12]:
from preprocessing.preprocessor import Preprocessor, Resample, Reorient, Normalize, CorrectX10, N4BiasFieldCorrection, SaveNifti

# ref image for reorientation
ref_img = nib.load(os.path.join('../example', 'RARE', 'TBI_fm_19_49', 'Anat', 'TBI_fm_19_49_N4.nii.gz'))

# Create an instance of the MRIProcessor class
processor = Preprocessor([
    CorrectX10(),
    N4BiasFieldCorrection(),
    SaveNifti(postfix='_N4', replace=input_postfix),
    Reorient(ref_img),
    Resample(target_resolution=patch_resolution, interpolation=0),
    Normalize()
])

## Final Inference for random cropping
Makes the predictions by sliding through the input a patch volume of size (76,76,76) with a stride of 8

In [1]:
from evaluation.inference import RandomCroppingPrediction

# Create an instance of the RandomCroppingPrediction class
predictor = RandomCroppingPrediction(model, patch_size=patch_size, stride=stride, threshold=threshold, num_classes=num_classes)


KeyboardInterrupt



Make predictions for every file inside the folder

In [14]:
def save_diff_mask(y_pred, gt, path):
    # Save a mask with only the voxels that are different between the prediction and the ground truth
    diff_mask = np.zeros_like(y_pred)
    diff_mask[y_pred != gt] = 1
    diff_mask = nib.Nifti1Image(diff_mask, affine=np.eye(4))
    nib.save(diff_mask, path)

In [15]:
import ants
import evaluation.postprocessing
importlib.reload(evaluation.postprocessing)
from evaluation.postprocessing import ipsi_contra_division_callback, morphology_refinement_callback
from preprocessing.preprocessor import ants_n4_bias_field_wrapper
from utils.nifti import create_3d_image_from_dti

for modality, folder in modalities_directories.items():

    # if modalities folder does not exist, skip
    if not os.path.isdir(os.path.join(dataset_folder, modality)): 
        print(f'Folder {folder} not found, skipping ({os.path.join(dataset_folder, modality)})')
        continue

    modality_folder = os.path.join(dataset_folder, modality)

    for case in os.listdir(modality_folder):

        # if not dir skip
        if not os.path.isdir(os.path.join(modality_folder, case)): continue
        
        # If case substring is not in the process_id list, skip
        if process_id and case not in process_id: continue
        
        if not os.path.isdir(os.path.join(modality_folder, case, folder)): print(f'Folder {folder} not found, skipping'); continue
        case_folder = os.path.join(modality_folder, case, folder)

        for file in os.listdir(case_folder):
            if file == case + input_postfix:
                print('\n|-',file, '-------------------\ \n')
                file_path = os.path.join(case_folder, file)

                # load image
                nii_img = nib.load(file_path)

                # If dti, process it with create_3d_image_from_dti
                if 'DTI' in folder.upper() and len(nii_img.shape) > 3:
                    name = case + '_dti_out'
                    nii_img = create_3d_image_from_dti(nii_img, output_path=case_folder, name=name)

                print(nii_img.shape, nii_img.header.get_zooms())
                x_prep = processor.preprocess(nii_img, path=file_path)

                # create a mask for this new unseen file
                results = predictor.random_cropping_inference(x_prep, with_brain_mask=True)
                y_mask = results['brain_mask']
                y_regions = results['roi']

                # postprocessing
                y_mask = morphology_refinement_callback(fill_small_holes=fill_small_holes, holes_max_area=holes_max_area,remove_small_objects=remove_small_objects, object_min_area=object_min_area)(y_mask)
                y_regions = ipsi_contra_division_callback(use_centroids=True)(y_regions)

                # save
                y_pred_nifti = nib.Nifti1Image(y_regions, affine=x_prep.affine, dtype=np.float64, header=x_prep.header)
                y_pred_mask_nifti = nib.Nifti1Image(y_mask, affine=x_prep.affine, dtype=np.float64, header=x_prep.header)

                if show_3d_preview: plot_slicer_cloud(x_prep, y_pred_nifti)

                # return to original shape and resolution
                save_path = os.path.join(save_folder, modality, case, folder, case + roi_prediction_postfix)
                mask_save_path = os.path.join(save_folder, modality, case, folder, case + brain_prediction_postfix)
                final_image = processor.deprocess(y_pred_nifti, nii_img, postprocessed_mapping, save_path=save_path, verbose=False)
                final_mask = processor.deprocess(y_pred_mask_nifti, nii_img, postprocessed_mapping, save_path=mask_save_path, verbose=False)
                estimate_volume(final_image, resolution=patch_resolution, verbose=True)

                # Save diff mask by loading 'Labels.nii.gz' and comparing it with the prediction
                # gt = nib.load(os.path.join(case_folder, case + '_Labels.nii.gz')).get_fdata()
                # b_gt = nib.load(os.path.join(case_folder, case + '_brain_mask.nii.gz')).get_fdata()
                # save_diff_mask(final_image.get_fdata(), gt, path= os.path.join(case_folder, case + '_diff_roi.nii.gz'))
                # save_diff_mask(final_mask.get_fdata(), b_gt, path= os.path.join(case_folder, case + '_diff_brain.nii.gz'))

                print ('----------------------------------------------------------// \n\n')

    # save an excel file with the volumes
    save_excel_table(os.path.join(dataset_folder, modality_folder), sub_folder=folder, include_only_list=None, save_folder=os.path.join(save_folder, modality_folder),
                    pred_roi_name=roi_prediction_postfix, pred_brain_name=brain_prediction_postfix, name_mapping=name_mapping, file_name=excel_name, postfix_mode=True)


|- TBI_biomech_fm_24__216_anat_orig.nii.gz -------------------\ 

(150, 150, 28) (0.099992745, 0.10000196, 0.29999766)
Number of classes:  5

- Classes present in the mask:  [ 0.  1.  3. 13. 21.]
Class 0.0 has 624882 voxels and a volume of 624.8820000000001 mm^3
Class 1.0 has 2217 voxels and a volume of 2.217 mm^3
Class 3.0 has 2242 voxels and a volume of 2.2420000000000004 mm^3
Class 13.0 has 184 voxels and a volume of 0.18400000000000005 mm^3
Class 21.0 has 475 voxels and a volume of 0.47500000000000003 mm^3
----------------------------------------------------------// 



|- TBI_biomech_fm_24__218_anat_orig.nii.gz -------------------\ 

(150, 150, 28) (0.09999978, 0.09999823, 0.29999697)
Number of classes:  5

- Classes present in the mask:  [ 0.  1.  3. 13. 21.]
Class 0.0 has 624975 voxels and a volume of 624.975 mm^3
Class 1.0 has 297 voxels and a volume of 0.2970000000000001 mm^3
Class 3.0 has 1615 voxels and a volume of 1.6150000000000002 mm^3
Class 13.0 has 2355 voxels and a vo

OSError: Cannot save file into a non-existent directory: '\calypso.marionegri.it\ImagingUserData\DannoCerebraleAcuto_MRI\Biomech_dec24'

In [22]:
import importlib
import utils.utils
importlib.reload(utils.utils)
from utils.utils import save_excel_table

save_excel_table(r"I:\\DannoCerebraleAcuto_MRI\\PNRR BATMAN Microbiota\\Exp SCFA", sub_folder="Anat", include_only_list=None, save_folder=r"I:\\DannoCerebraleAcuto_MRI\\PNRR BATMAN Microbiota\\Exp SCFA",
                    pred_roi_name=roi_prediction_postfix, pred_brain_name=brain_prediction_postfix, name_mapping=name_mapping, file_name=excel_name, postfix_mode=True)

Processing 47 subjects...
I:\\DannoCerebraleAcuto_MRI\\PNRR BATMAN Microbiota\\Exp SCFA\TBI_PNRR_BATMAN_fp_24_101\Anat\TBI_PNRR_BATMAN_fp_24_101_regions_r3dnet.nii.gz
I:\\DannoCerebraleAcuto_MRI\\PNRR BATMAN Microbiota\\Exp SCFA\TBI_PNRR_BATMAN_fp_24_102\Anat\TBI_PNRR_BATMAN_fp_24_102_regions_r3dnet.nii.gz
I:\\DannoCerebraleAcuto_MRI\\PNRR BATMAN Microbiota\\Exp SCFA\TBI_PNRR_BATMAN_fp_24_104\Anat\TBI_PNRR_BATMAN_fp_24_104_regions_r3dnet.nii.gz
I:\\DannoCerebraleAcuto_MRI\\PNRR BATMAN Microbiota\\Exp SCFA\TBI_PNRR_BATMAN_fp_24_105\Anat\TBI_PNRR_BATMAN_fp_24_105_regions_r3dnet.nii.gz
I:\\DannoCerebraleAcuto_MRI\\PNRR BATMAN Microbiota\\Exp SCFA\TBI_PNRR_BATMAN_fp_24_106\Anat\TBI_PNRR_BATMAN_fp_24_106_regions_r3dnet.nii.gz
I:\\DannoCerebraleAcuto_MRI\\PNRR BATMAN Microbiota\\Exp SCFA\TBI_PNRR_BATMAN_fp_24_107\Anat\TBI_PNRR_BATMAN_fp_24_107_regions_r3dnet.nii.gz
I:\\DannoCerebraleAcuto_MRI\\PNRR BATMAN Microbiota\\Exp SCFA\TBI_PNRR_BATMAN_fp_24_108\Anat\TBI_PNRR_BATMAN_fp_24_108_regions_r